In [1]:
import os
import sys
import json
import cv2
import torch
import numpy as np
import matplotlib.pyplot as plt

from pathlib import Path
from PIL import Image
from torch.utils.data import random_split, DataLoader, ConcatDataset
import torch.nn as nn
import segmentation_models_pytorch as smp

# ─────────────────────────────────────────────────────────────────────────────
# 1. Повторяем подготовку данных
# ─────────────────────────────────────────────────────────────────────────────
seminar_path = "/home/alexskv/seminar_2"
seminar_path = Path(seminar_path)
data_path = f"{seminar_path}/data"
sys.path.insert(0, f"{seminar_path}/core")
print(sys.path)

from dataset import SimpleCocoDataset
from trainers import SimpleSegmentationTrainer

#Определяем параметры даталодера 
resize = (512, 512)
batch_size = 32

pathology_ids = [i for i in range(6, 27) if i != 15]


#Создаем классы, так же берем только патологию.
out_classes = [{"id": 1, "name": "Патология", "summable_masks": pathology_ids,    "subtractive_masks": []}]

base_names = [
    "Правое лёгкое", "Левое лёгкое", "Контуры сердца", "Купола диафрагмы и нижележащая область",
    "Сложный случай", "нельзя составить заключение", "Иная патология", "Гидроторакс",
    "Легочно-венозная гипертензия 2 стадии и выше", "Пневмоторакс", "Доброкачественное новообразование",
    "Перелом ребра свежий", "Буллезное вздутие, тонкостенная киста", "Рак лёгкого (включая дорожку к корню при наличии)",
    "Кардиомегалия (отмечается всё сердце, как патология)", "Интерстициальная пневмония.",
    "Метастатическое поражение лёгких", "Полость с уровнем жидкости", "Грыжа пищевого отверстия диафрагмы",
    "Спавшийся сегмент лёгкого при ателектазе", "Инфильтративный туберкулёз",
    "Пневмония. В том числе сегментарная и полисегментарная", "Область распада, деструкции тканей лёгкого",
    "Участок пневмофиброза", "Кальцинаты. Каждый кальцинат выделяется отдельным контуром",
    "Консолидированный перелом ребра"
]

base_classes = [{"id": i+1, "name": name} for i, name in enumerate(base_names)]


data_roots = {p.parent.parent for p in (seminar_path / "data").rglob("annotations/instances_default.json")}
print(f"Найдено {len(data_roots)} датасетов:", *data_roots, sep="\n  ")


# ─────────────────────────────────────────────────────────────────────────────
# 2. Сбор и разделение данных
# ─────────────────────────────────────────────────────────────────────────────
datasets = [SimpleCocoDataset(str(d), base_classes, out_classes, resize=resize)
            for d in sorted(data_roots)]

full_ds = ConcatDataset(datasets)
val_percent = 0.2
val_size = int(len(full_ds) * val_percent)
train_size = len(full_ds) - val_size
train_ds, val_ds = random_split(full_ds, [train_size, val_size])

train_loader = DataLoader(train_ds,
                          batch_size=batch_size,
                          shuffle=True,
                          num_workers=4)

val_loader = DataLoader(val_ds,
                        batch_size=batch_size,
                        shuffle=False,
                        num_workers=4)


# ─────────────────────────────────────────────────────────────────────────────
# 3. Модель сегментации на базе Unet + EfficientNet-b0 encoder
# ─────────────────────────────────────────────────────────────────────────────
model = smp.Unet(
    encoder_name="efficientnet-b0",
    encoder_weights=None,
    in_channels=1,
    classes=1  # фон, норма, патология
)


# ─────────────────────────────────────────────────────────────────────────────
# 4. Запуск обучения сегментации
# ─────────────────────────────────────────────────────────────────────────────
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

trainer = SimpleSegmentationTrainer(
    model=model,
    classes = out_classes,
    train_loader=train_loader,
    val_loader=val_loader,
    device=device,
    epochs=10,
    exp_name="efficientnet_segmentation"
)

trainer.train()

['/home/alexskv/seminar_2/core', '/home/alexskv/seminar_2', '/usr/lib/python310.zip', '/usr/lib/python3.10', '/usr/lib/python3.10/lib-dynload', '', '/home/alexskv/segm-env/lib/python3.10/site-packages']


2025-04-14 11:56:35.643614: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-14 11:56:35.658240: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-14 11:56:35.662526: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-14 11:56:35.676029: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-14 11:56:36.372679: W tensorflow/compiler/tf2

Найдено 4 датасетов:
  /home/alexskv/seminar_2/data/task1
  /home/alexskv/seminar_2/data/task4
  /home/alexskv/seminar_2/data/task2
  /home/alexskv/seminar_2/data/task3
loading annotations into memory...
Done (t=0.03s)
creating index...
index created!
loading annotations into memory...
Done (t=0.03s)
creating index...
index created!
loading annotations into memory...
Done (t=0.06s)
creating index...
index created!
loading annotations into memory...
Done (t=0.02s)
creating index...
index created!


Train:  23%|█████████████████▍                                                           | 7/31 [00:16<00:57,  2.41s/it]

KeyboardInterrupt

